# Intermediate Neural Network in Keras

Bulding an Intermediate Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [1]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [2]:
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LeakyReLU
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


#### Load data

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#### Preprocess data
Flatten and normalise input data.

In [4]:
X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')

In [5]:
X_train /= 255
X_test /= 255

In [6]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### ReLUs

In [7]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

class ReLUs(Activation):
    
    def __init__(self, activation, **kwargs):
        super(ReLUs, self).__init__(activation, **kwargs)
        self.__name__ = 'relus'

def relus(Z):
    e_param = 0.001 #the best so far -> 0.0085
    pi = K.variable((3.14))
    m = e_param * (K.sigmoid(K.sin(Z)) - K.sigmoid(K.cos(Z)) * K.exp(K.sqrt(pi)))
    A = K.maximum(m, Z)
    return A

get_custom_objects().update({'relus': ReLUs(relus)})

#### Design Neural Network architecture

In [8]:
model = Sequential()
model.add(Dense(128, activation = 'linear', kernel_initializer='glorot_uniform', input_shape = (784,)))
model.add(LeakyReLU(alpha=0.0085))
model.add(Dropout(0.2))
model.add(Dense(256, activation = 'linear', kernel_initializer='glorot_uniform'))
model.add(LeakyReLU(alpha=0.0085))
model.add(Dropout(0.3))
model.add(Dense(1024, activation = 'linear', kernel_initializer='glorot_uniform'))
model.add(LeakyReLU(alpha=0.0085))
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              263168    
__________

#### Callbacks

In [10]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath='model_output/weights-deepnet-mnist.hdf5',
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=20)


if not os.path.exists('model_output'):
    os.makedirs('model_output')
    
tensorboard = TensorBoard("../logs/deepnet-mnist-leaky-relu-II")

#### Configure model

In [11]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### Train!

In [12]:
model.fit(X_train, y_train, batch_size = 128, epochs = 50, verbose = 1,
          validation_split = 0.1, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 6s 110us/step - loss: 0.3950 - acc: 0.8771 - val_loss: 0.1162 - val_acc: 0.9652
Epoch 2/50
54000/54000 [==============================] - 6s 102us/step - loss: 0.1767 - acc: 0.9471 - val_loss: 0.0972 - val_acc: 0.9708
Epoch 3/50
54000/54000 [==============================] - 5s 101us/step - loss: 0.1358 - acc: 0.9588 - val_loss: 0.0823 - val_acc: 0.9745
Epoch 4/50
54000/54000 [==============================] - 6s 109us/step - loss: 0.1142 - acc: 0.9652 - val_loss: 0.0817 - val_acc: 0.9757
Epoch 5/50
54000/54000 [==============================] - 5s 90us/step - loss: 0.0992 - acc: 0.9702 - val_loss: 0.0773 - val_acc: 0.9760
Epoch 6/50
54000/54000 [==============================] - 5s 91us/step - loss: 0.0876 - acc: 0.9731 - val_loss: 0.0722 - val_acc: 0.9793
Epoch 7/50
54000/54000 [==============================] - 5s 93us/step - loss: 0.0815 - acc: 0.9743 - val_loss: 0.0594 - val_

#### Test Predictions

In [13]:
saved_model = keras.models.load_model('model_output/weights-deepnet-mnist.hdf5')
predictions = saved_model.predict_classes(X_test, verbose = 2)
print(predictions)

[7 2 1 ..., 4 5 6]


#### Test Final Accuracy

In [16]:
final_loss, final_acc = saved_model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

10000/10000 [==============================] - 1s 68us/step
Final loss: 0.0682, final accuracy: 0.9833


In [15]:
#Final loss: 0.0632, final accuracy: 0.9837 -> relus (0.0085)
#Final loss: 0.0749, final accuracy: 0.9835 -> relu